Packages needed:

    * RigidBodyDynamics
    * MeshCatMechanisms
    * Optim
    * IntervalOptimisation
    * IntervalArithmetic
    * SymPy

In [ ]:
using RigidBodyDynamics
using MeshCatMechanisms
using SymPy: @syms

In [ ]:
# Load the description of the mechanism from its URDF file 
urdf = "kuka_iiwa.urdf"
mechanism = parse_urdf(Float64, urdf)

# Create a visualizer to show the basic joint skeleton
# and mass distribution of the robot, and open it in a 
# new browser tab
vis = MechanismVisualizer(mechanism, Skeleton(randomize_colors=true))
open(vis)

In [ ]:
# Given a mechanism, a point located somewhere on that mechanism,
# and a target point (on that mechanism or fixed in the world),
# create a cost function which maps the configuration vector
# of the robot (the position of every joint) to the distance
# from the given point to the target
function inverse_kinematics_problem(mechanism::Mechanism, 
        point_on_body::Point3D, target::Point3D)
    let statecache = StateCache(mechanism), point = point_on_body, target = target
        function cost(configuration)
            state = statecache[eltype(configuration)]
            set_configuration!(state, configuration)
            error = transform(state, point_on_body, target.frame) - target
            norm(error.v)
        end
    end
end

In [ ]:
# Create a point on the `iiwa_link_ee` body (the end of the arm)
point = Point3D(default_frame(findbody(mechanism, "iiwa_link_ee")), 0., 0, 0)
# Create a target location in world frame
target = Point3D(root_frame(mechanism), 0.5, 0.5, 0.5)

# Visualize both points as small spheres in the 3D viewer
setelement!(vis, point, 0.01)
setelement!(vis, target, 0.02)

# Create the cost function
cost = inverse_kinematics_problem(mechanism, point, target)

In [ ]:
# We can use SymPy to get an exact symbolic expression for the function
# we're trying to minimize: 
@syms q1 q2 q3 q4 q5 q6 q7 real=true

c_symbolic = cost([q1, q2, q3, q4, q5, q6, q7])

# Displaying that symbolic expression in Jupyter is pretty slow, so 
# let's just write it out to a LaTeX file
open("cost.tex", "w") do f
    show(f, "text/latex", c_symbolic)
end

In [ ]:
# First, let's get a (potentially local) optimal solution with Optim
using Optim

N = num_positions(mechanism) # number of joint position variables
f = Optim.OnceDifferentiable(cost, zeros(N), autodiff=:forward)
result = Optim.optimize(f, zeros(N), BFGS())

In [ ]:
# Render the robot at the resulting configuration. The tip of the arm
# should match the target pose
set_configuration!(vis, Optim.minimizer(result))

In [ ]:
# Now do the same thing with IntervalOptimisation
using IntervalOptimisation, IntervalArithmetic

In [ ]:
r = IntervalOptimisation.minimize(cost, 
    IntervalBox(-10..10, N),
    1e-2)